<img src="../Img/banner-fa-49-2.jpg">

### <center> Финансовый университет
## <center>  Современные технологии <br>прикладного программирования и обработки данных
## <center> Тема 6. Методы группировки данных

20 апреля 2021 года

Лекция<br>
Студенты: Поток У18-7у<br>
Преподаватель: Смирнов Михаил Викторович, доцент Департамента Анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации

# Группировка данных

Данные можно представить себе как последовательность каких-то единичных наблюдений. Например, в наборе данных *FinancialSample2.csv* единичное наблюдение — это сделка по продаже конкретного товара.

In [2]:
import pandas as pd
fs=pd.read_csv("./Data/FinancialSample2.csv", sep=';')
fs.head()

,Id,Segment,Country,Product,DiscountBand,UnitsSold,ManufacturingPrice,SalePrice,GrossSales,Discounts,Sales,COGS,Profit,Date,MonthNumber,MonthName,Year
0,0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.0,2014-01-01,1.0,January,2014.0
1,1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.0,2014-01-01,1.0,January,2014.0
2,2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.0,2014-06-01,6.0,June,2014.0
3,3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.0,2014-06-01,6.0,June,2014.0
4,4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.0,2014-06-01,6.0,June,2014.0


У каждого наблюдения есть какие-то атрибуты. Они могут быть разных типов:

- категориальные — например, название фирмы-производителя товара (признак *Product*);
- численные — например, размер партии товара в штуках (признак *UnitsSold*).

Решение задачи группировки предполагает разделение данных по некоторому признаку (атрибуту), после чего к каждому элементу этих разделенных данных мы можем применить агрегирующую операцию. Это такая операция, которая позволяет вычислить какой-либо показатель.

Затем мы можем оценить, как отличаются эти показатели в зависимости от признака, по которому было осуществлено разделение. Такое разделение мы можем назвать группировкой данных.

Возьмем информацию о числе единиц проданного товара в пяти первых сделках для стран Пакистан, Германия, Мексика.

In [3]:
fs_pakistan = fs[fs["Country"]=="Pakistan"][["Country", "UnitsSold"]].head()
fs_germany = fs[fs["Country"]=="Germany"][["Country", "UnitsSold"]].head()
fs_mexico = fs[fs["Country"]=="Mexico"][["Country", "UnitsSold"]].head()

Найдем среднее значение *UnitsSold* в каждой группе

In [4]:
print(fs_pakistan["UnitsSold"].mean())
print(fs_germany["UnitsSold"].mean())
print(fs_mexico["UnitsSold"].mean())

2485.0
1237.6
1551.0


Объединим три этих таблицы в одну

In [5]:
fs_joined = pd.concat([fs_pakistan, fs_germany, fs_mexico], axis=0, ignore_index=True)
fs_joined

,Country,UnitsSold
0,Pakistan,2911.0
1,Pakistan,1513.0
2,Pakistan,2572.0
3,Pakistan,2552.0
4,Pakistan,2877.0
5,Germany,1321.0
6,Germany,888.0
7,Germany,1513.0
8,Germany,921.0
9,Germany,1545.0


Применим функцию *groupby()*

In [6]:
fs_joined.groupby(by="Country")["UnitsSold"].mean()

Country
Germany     1237.6
Mexico      1551.0
Pakistan    2485.0
Name: UnitsSold, dtype: float64

## Функция *groupby()*

Ранее мы уже рассмотрели простейшую группировку с помощью функции *value_counts()*, которая возвращает серию, содержащую количество уникальных значений в группах. Кратко вспомним, как работает *value_counts()* на примере. Пусть имеются сведения об успеваимости студентов: имя студента, учебная группа, оценка в баллах в первом и втором семестрах.

In [7]:
students=pd.DataFrame({'Name':
                    ['Иван','Петр','Мария','Василий','Анна','Василиса','Дарья','Николай','Вероника'],
                    'Group':['Ф1','Ф1','Ф2','Ф2','Ф1','Ф2','Ф2','Ф2','Ф1'],
                    'Mark_1':[19,18,17,21,24,18,22,19,18],
                    'Mark_2':[25,30,26,26,21,23,25,29,31]})

Распечатаем датафрейм с данными о студентах, предварительно добавив столбец - суммарную оценку

In [8]:
students['Mark_sum'] = students['Mark_1'] + students['Mark_2']
students

,Name,Group,Mark_1,Mark_2,Mark_sum
0,Иван,Ф1,19,25,44
1,Петр,Ф1,18,30,48
2,Мария,Ф2,17,26,43
3,Василий,Ф2,21,26,47
4,Анна,Ф1,24,21,45
5,Василиса,Ф2,18,23,41
6,Дарья,Ф2,22,25,47
7,Николай,Ф2,19,29,48
8,Вероника,Ф1,18,31,49


Применим *value_counts()* к столбцам *Group* и *Mark_1*

In [9]:
students['Group'].value_counts()

Ф2    5
Ф1    4
Name: Group, dtype: int64

Результат: уникальными значениями столбца *Group* являются Ф1 и Ф2, которые встречаются 4 и 5 раз соответственно. Применим *value_counts()* к численному столбцу *Mark_1*.

In [10]:
students['Mark_1'].value_counts()

18    3
19    2
24    1
22    1
21    1
17    1
Name: Mark_1, dtype: int64

Функция *value_counts()* сгруппировала студентов по группам и по оценкам. Предположим теперь, что нас интересует не количество студентов, а сумма баллов. Это можно сделать с помощью известного нам уже способа отбора по условию и применения статистической функции суммы. Для первого семестра получим:

In [11]:
print(students[students['Group']=='Ф1']['Mark_1'].sum())
print(students[students['Group']=='Ф2']['Mark_1'].sum())

79
97


Для таких целей удобнее применять функцию *groupby()*

In [12]:
students.groupby(['Group'])['Mark_1'].sum()

Group
Ф1    79
Ф2    97
Name: Mark_1, dtype: int64

Рассмотрим более подробно работу функции *groupby()*. Выполним команду `students.groupby(['Group'])`

In [13]:
students.groupby(['Group'])

Мы получили объект, который хранит сведения о том, какие строки относятся к каким значениям группируемого столбца. Чтобы распечатать эти сведения выполним эту же команду с атрибутом `.groups`

In [14]:
students.groupby(['Group']).groups

{'Ф1': Int64Index([0, 1, 4, 8], dtype='int64'),
 'Ф2': Int64Index([2, 3, 5, 6, 7], dtype='int64')}

Теперь мы узнали, что к группе Ф1 относятся строки [0, 1, 4, 8], а к Ф2 [2, 3, 5, 6, 7]. Эту информацию использует `pandas` для того, чтобы разбить данные на подгруппы и в каждой подгруппе рассчитать что-либо для нас.

Для того, чтобы провести суммирование (или выполнить другую агрегирующую функцию) по нескольким столбцам, их надо явно указать. Проведем суммирование баллов в каждой учебной группе отдельно в первом и во втором семестрах.

In [15]:
students_groupped = students.groupby(['Group'])[['Mark_1','Mark_2']].sum()
students_groupped

,Mark_1,Mark_2
Group,,
Ф1,79,107
Ф2,97,129


Объект *studens_groupped* - это *DataFrame*.

In [16]:
type(students_groupped)

pandas.core.frame.DataFrame

Как всегда, мы можем получить доступ к его элементам. Например, чтобы получить сумму баллов студентов Ф2 во втором семестре введем команду

In [17]:
students_groupped.loc['Ф2','Mark_2']

129

Если не уточнять, по какому столбцу проводится суммирование (или другая агрегирующая функция), то получим сумму по всем числовым столбцам

In [18]:
students_groupped = students.groupby(['Group']).sum()
students_groupped

,Mark_1,Mark_2,Mark_sum
Group,,,
Ф1,79,107,186
Ф2,97,129,226


Студенты какой группы получили в сумме больше баллов? Для ответа отсортируем с помощью `.sort_values()`  таблицу сгруппированных значений по убыванию суммы баллов. Это группа Ф2.

In [19]:
students_groupped.sort_values('Mark_sum',ascending=False)

,Mark_1,Mark_2,Mark_sum
Group,,,
Ф2,97,129,226
Ф1,79,107,186


## Другие агрегирующие функции

Еще раз распечатаем набор данных

In [20]:
fs.head()

,Id,Segment,Country,Product,DiscountBand,UnitsSold,ManufacturingPrice,SalePrice,GrossSales,Discounts,Sales,COGS,Profit,Date,MonthNumber,MonthName,Year
0,0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.0,2014-01-01,1.0,January,2014.0
1,1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.0,2014-01-01,1.0,January,2014.0
2,2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.0,2014-06-01,6.0,June,2014.0
3,3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.0,2014-06-01,6.0,June,2014.0
4,4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.0,2014-06-01,6.0,June,2014.0


### Функции *max(), min()*

Функция *max()* подсчитывает максимальное значение в серии. Найдем максимальную скидку с группировкой по стране.

In [21]:
fs.groupby("Country")[["Discounts"]].max()

,Discounts
Country,
Albania,22.48
Canada,119756.00
Egypt,22.67
Finland,22.48
France,111375.00
Germany,106512.00
Mexico,149677.50
Pakistan,22.67
United States of America,125820.00


Функция *min()* подсчитывает минимальное значение в серии. Найдем минимальное число единиц  товара с группировкой по сегменту рынка и отсортируем по убыванию.

In [22]:
fs.groupby('Segment')['UnitsSold'].min().sort_values(ascending=False)

Segment
Agriculture         1316.0
Science             1256.0
Household           1255.0
Enterprise           330.0
Channel Partners     306.0
Midmarket            218.0
Small Business       214.0
Government           200.0
Name: UnitsSold, dtype: float64

### Функция *nunique()*

Функция *nunique()* позволяет посчитать количество уникальных значений в серии. Её лучше всего применять к столбцам, в которых хранятся категориальные данные.

In [23]:
fs.groupby("Country")[["Segment", "Product", "DiscountBand"]].nunique()

,Segment,Product,DiscountBand
Country,,,
Albania,8,8,5
Canada,8,9,6
Egypt,8,8,5
Finland,8,8,5
France,5,6,4
Germany,8,9,6
Mexico,8,9,6
Pakistan,8,8,5
United States of America,8,9,6


### Функция *count()*

Позволяет посчитать количество непустых элементов в группе. Найдём число сделок в каждом сегменте. Число непустых значений будем подсчитывать по столбцу *Id*. Если конкретный столбец явно не указать, то *pandas* выведет на печать все столбцы таблицы.

In [24]:
fs.groupby('Segment')['Id'].count()

Segment
Agriculture          85
Channel Partners    198
Enterprise          190
Government          389
Household            70
Midmarket           178
Science              88
Small Business      202
Name: Id, dtype: int64

Аналогичный результат получим с помощью *value_counts()*. Сравните:

In [25]:
fs['Segment'].value_counts()

Government          389
Small Business      202
Channel Partners    198
Enterprise          190
Midmarket           178
Science              88
Agriculture          85
Household            70
Name: Segment, dtype: int64

### Функция *median()*

Находит медианное значение. Рассчитаем среднее и медианное значение прибыли по сделкам каждого производителя. Для этого воспользуемся функцией *.agg()*

In [26]:
fs.groupby('Product')['Profit'].agg(['mean','median'])

,mean,median
Product,,
Amarilla,29937.277234,10911.9000
Bal,3177.772562,2937.6000
Carretera,11700.684866,3785.1180
Geotrack,3213.240861,3140.4780
Montana,13375.409455,3669.0080
Nice,3032.521568,2755.6235
Paseo,17675.339098,4436.8500
VTT,16802.487679,3932.9500
Velo,12792.886549,3560.8795


### Функция *std()*
Рассчитывает стандартное отклонение. Найдем среднее, медианное значения и стандартное отклонение себестоимости и прибыли от продажи товара в каждом сегменте.

In [27]:
fs.groupby('Segment')[['COGS', 'Profit']].agg(['mean','median','std'])

COGS                                 Profit  \
                           mean    median            std          mean   
Segment                                                                  
Agriculture        10600.100000  10623.20    2120.259169   2922.329200   
Channel Partners    7566.898485   7578.35    3562.078901   8236.115288   
Enterprise        111395.994211  66360.00  124914.244317  -1769.628926   
Government        108070.419280  12660.00  194388.604173  29995.488933   
Household          10628.760000  10385.90    2056.484196   3146.672071   
Midmarket          14472.774157  12460.00    7509.130571   4983.232983   
Science            10408.586364  10550.40    1902.168898   3223.880773   
Small Business    194919.771782  13902.35  240965.024680  22076.607757   

                                           
                     median           std  
Segment                                    
Agriculture       2641.8000    950.974586  
Channel Partners  4752.0015   6952.042483  
Enterprise        2293.1640   9780.839737  
Government        5166.0960  51387.453473  
Household         3080.9780   1140.580805  
Midmarket         3874.3850   3296.928288  
Science           3166.4285   1103.447176  
Small Business    5134.7140  29059.520725

## Сводные таблицы

Что такое "сводная таблица" и  чем она отличается от "плоской" таблицы? Как правило, значениями плоской таблицы являются атрибуты параметров объектов, расположенных в строках. Названия параметров расположены в столбцах. Например, каждая строка *Financial Sample* является объектом - сделкой по продаже товара. Параметры сделки указаны в столбцах. на пересечении строки и столбца содержится значение параметра для конкретного объекта. 

Значениями сводной таблицы является результат вычисления агрегирующей функции - сгруппированные данные. При этом показатели, на основе которых происходит объединение данных, расположены в заголовках строк и столбцов.

### Функция *pivot_table()*

Функция *pivot_table()* позволяет быстро и просто составлять сводные таблицы
Согласно документации https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html 

```
pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
```

параметрами функции являются:

- data - исходная таблица с данными
- index - строки
- columns - столбцы
- values - агрегируемый столбец исходной таблицы
- aggfunc - функция агрегации
- margins - вывод суммирующих строки и столбца (логическое значение)
- fill_value - значение, на которое следует заменять пустые значения
- dropna - удалять или нет результаты, в которых все значения пустые (логическое)

Составим сводную таблицу, в которой отобразим макимальную скидку, предоставленную каждым производителем в каждом сегменте рынка. Расположим названия сегментов по строкам, а производителей по столбцам. В качестве агрегируемого столбца укажем *Discounts* -  столбец, в котором содержится значение скидки. Функция агрегации "max".

In [59]:
pd.pivot_table(fs, values='Discounts', index='Segment', columns=['Product'], aggfunc='max')

Product,Amarilla,Bal,Carretera,Geotrack,Montana,Nice,Paseo,VTT,Velo
Segment,,,,,,,,,
Agriculture,NaN,22.10,20.21,21.72,21.34,19.08,20.78,22.29,18.320
Channel Partners,4895.520,22.10,2556.84,22.48,3831.84,19.08,5314.32,3250.80,2124.360
Enterprise,49770.000,22.48,51881.25,22.48,31466.25,21.72,33563.75,55387.50,45712.500
Government,78400.000,19.64,81445.00,21.34,109147.50,21.16,149677.50,98245.00,119756.000
Household,NaN,19.27,22.48,22.29,22.67,22.10,18.70,22.67,21.530
Midmarket,5279.175,22.48,5005.65,22.29,4830.00,22.67,5757.75,6974.10,7795.125
Science,NaN,22.67,22.29,22.29,22.29,20.59,22.48,21.72,22.480
Small Business,111375.000,17.76,92763.00,22.67,102667.50,22.48,125820.00,106722.00,115830.000


Добавим для производителей уровень группирвки по стране. Для экономии места и большей наглядности ограничимся двумя странами - Германией и Францией.

In [60]:
fs_GF = fs[fs['Country'].isin(['Germany','France'])]

In [62]:
pd.pivot_table(fs_GF, values='Discounts', 
               index='Segment', columns=['Country','Product'], aggfunc='max')

Country              France                                          \
Product            Amarilla Carretera   Montana     Paseo       VTT   
Segment                                                               
Agriculture             NaN       NaN       NaN       NaN       NaN   
Channel Partners    4158.00   1581.36   1967.28   3201.66   2412.72   
Enterprise         19703.75  33563.75  20891.25  33563.75   2180.00   
Government         70462.00  81445.00  52479.00  94178.00  62769.00   
Household               NaN       NaN       NaN       NaN       NaN   
Midmarket           3108.00    588.00   3108.00   3420.90   6974.10   
Science                 NaN       NaN       NaN       NaN       NaN   
Small Business    111375.00  45801.00  63828.00  35748.00  21978.00   

Country                       Germany                                      \
Product                Velo  Amarilla    Bal Carretera Geotrack   Montana   
Segment                                                                     
Agriculture             NaN       NaN  12.66     14.36    21.72     16.24   
Channel Partners   2124.360   3088.80  13.22   1706.40      NaN   1405.20   
Enterprise        12431.250  43518.75  15.68  51881.25      NaN  21875.00   
Government        43596.000  26698.00  19.08  30492.00    14.73  58751.00   
Household               NaN       NaN  11.90     15.87      NaN     22.48   
Midmarket          7795.125   1309.50  20.02   3177.30    22.29    853.20   
Science                 NaN       NaN    NaN     17.94      NaN     22.29   
Small Business    34839.000  18750.00    NaN  92763.00      NaN  24252.00   

Country                                                 
Product            Nice     Paseo       VTT       Velo  
Segment                                                 
Agriculture         NaN       NaN       NaN      13.79  
Channel Partners  14.92   1917.00   1860.60    1580.28  
Enterprise        20.59  30738.75  10350.00   18261.25  
Government        14.92  60088.00  65450.00   41170.50  
Household         22.10      9.44       NaN      14.36  
Midmarket         22.67   2643.75   2643.75    1377.00  
Science           17.57       NaN     12.66      21.72  
Small Business      NaN  48924.00  71793.00  106512.00

### Доступ к элементам сводной таблицы

Как получить доступ к отдельному столбцу или столбцам?

In [63]:
pvt=pd.pivot_table(fs_GF, values='Discounts', 
               index='Segment', columns=['Country','Product'], aggfunc='max')

In [64]:
type(pvt)

pandas.core.frame.DataFrame

In [66]:
pvt.columns

MultiIndex([( 'France',  'Amarilla'),
            ( 'France', 'Carretera'),
            ( 'France',   'Montana'),
            ( 'France',     'Paseo'),
            ( 'France',       'VTT'),
            ( 'France',      'Velo'),
            ('Germany',  'Amarilla'),
            ('Germany',       'Bal'),
            ('Germany', 'Carretera'),
            ('Germany',  'Geotrack'),
            ('Germany',   'Montana'),
            ('Germany',      'Nice'),
            ('Germany',     'Paseo'),
            ('Germany',       'VTT'),
            ('Germany',      'Velo')],
           names=['Country', 'Product'])

In [67]:
pvt.index

Index(['Agriculture', 'Channel Partners', 'Enterprise', 'Government',
       'Household', 'Midmarket', 'Science', 'Small Business'],
      dtype='object', name='Segment')

In [68]:
pvt.loc['Channel Partners', ( 'France',  'Amarilla')]

4158.0

## Консолидация: объединение данных с помощью сводной таблицы

Задача.
Имеются csv-файлы с информацией об изменении цены акций. 



1.   [GAZA](https://drive.google.com/file/d/1bw3d01WXw5_N1skmnXwZUHFeT_xTiU1c/view?usp=sharing)
2.   [KMAZ](https://drive.google.com/file/d/1JKendyxL4tF4-JrA7DzkHKzgSYjHtVkT/view?usp=sharing)
3.   [ROSN](https://drive.google.com/file/d/18ImO_nT9yaghfkLfe3UNkmqfwszZI1Si/view?usp=sharing)

*Источник: [finam.ru](https://www.finam.ru/)*

Необходимо объединить эти данные в одну таблицу, в строках разместить дату, в столбцах код компании, в ячейках цену акции в соответствующий день.

In [3]:
GAZA=pd.read_csv('./Data/GAZA_200401_201130.csv',sep=';')
KMAZ=pd.read_csv('./Data/KMAZ_200401_201130.csv',sep=';')
ROSN=pd.read_csv('./Data/ROSN_200401_201130.csv',sep=';')

Ознакомимся кратко с этими источниками, отобразив первые пять строк.

In [4]:
GAZA.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>
0,GAZA,W,06/04/20,0,375.5
1,GAZA,W,13/04/20,0,386.5
2,GAZA,W,20/04/20,0,396.5
3,GAZA,W,27/04/20,0,387.0
4,GAZA,W,04/05/20,0,382.0


In [5]:
KMAZ.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>
0,KMAZ,W,06/04/20,0,54.1
1,KMAZ,W,13/04/20,0,53.0
2,KMAZ,W,20/04/20,0,54.9
3,KMAZ,W,27/04/20,0,55.5
4,KMAZ,W,04/05/20,0,56.0


In [6]:
ROSN.head()

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>
0,ROSN,W,06/04/20,0,344.00
1,ROSN,W,13/04/20,0,313.90
2,ROSN,W,20/04/20,0,324.00
3,ROSN,W,27/04/20,0,335.65
4,ROSN,W,04/05/20,0,348.00


Объединение таблиц с помощью <TT>.concat()</tt>

In [9]:
# "Объединение" таблиц проводится с помощью .concat()
join=pd.concat([GAZA,KMAZ,ROSN], axis=0)

# в объединенной таблице преобразуем дату к типу даты
join['<DATE>']=pd.to_datetime(join['<DATE>'],dayfirst=True) 

# и перестраиваем индекс
join.index=range(len(join)) 
join

,<TICKER>,<PER>,<DATE>,<TIME>,<CLOSE>
0,GAZA,W,2020-04-06,0,375.50
1,GAZA,W,2020-04-13,0,386.50
2,GAZA,W,2020-04-20,0,396.50
3,GAZA,W,2020-04-27,0,387.00
4,GAZA,W,2020-05-04,0,382.00
...,...,...,...,...,...
97,ROSN,W,2020-10-26,0,349.80
98,ROSN,W,2020-11-02,0,382.35
99,ROSN,W,2020-11-09,0,419.35
100,ROSN,W,2020-11-16,0,464.80


Проверка по числу строк и столбцов.

In [10]:
# Проверка. В объединенной таблице столько же столбцов, сколько в любой из исходных таблиц 
# и столько строк, сколько в сумме у всех объединяесыъ таблиц.
print(GAZA.shape)
print(KMAZ.shape)
print(ROSN.shape)
print(join.shape)

(34, 5)
(34, 5)
(34, 5)
(102, 5)


Краткая информация об объединенной таблице

In [11]:
join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   <TICKER>  102 non-null    object        
 1   <PER>     102 non-null    object        
 2   <DATE>    102 non-null    datetime64[ns]
 3   <TIME>    102 non-null    int64         
 4   <CLOSE>   102 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 4.1+ KB


Создадим сводную таблицу

In [14]:
pvt=join.pivot_table(index='<DATE>', columns='<TICKER>', values='<CLOSE>')
pvt

<TICKER>,GAZA,KMAZ,ROSN
<DATE>,,,
2020-04-06,375.5,54.1,344.00
2020-04-13,386.5,53.0,313.90
2020-04-20,396.5,54.9,324.00
2020-04-27,387.0,55.5,335.65
2020-05-04,382.0,56.0,348.00
2020-05-11,377.5,54.3,350.05
2020-05-18,387.0,55.0,364.50
2020-05-25,381.0,55.9,376.20
2020-06-01,389.5,60.0,401.95


Проведена консолидация данных: объединенная таблица содержит цену акций из всех трех источников и только в одном столбце момент времени.

Проверка. Подсчитаем число строк в консолидированной таблице. Оно совпадает с числом строк в любой из исходных таблиц.

In [18]:
len(pvt)

34

## Выполните контрольные задания

### 1. Исходный набор данных

Загрузите в pandas.DataFrame набор данных *ListungAm.csv* из папки *../Data*. Загрузите файл *Description.csv* с описанием полей этого набора данных.

### 2. Выполните задания
1. Найдите среднее число коек в каждом районе Амстердама.

In [20]:
# Ваш код здесь


2. Найдите среднюю оценку чистоты (качества уборки) помещения для каждого типа собственности.

In [21]:
# Ваш код здесь


3. Найдите среднюю цену размещения в для каждого типа комнаты в районе *Westerpark*.

In [22]:
# Ваш код здесь


4. С помощью *groupby()* найдите количество объектов размещения для каждого типа собственности в *Noord-West*.

In [ ]:
# Ваш код здесь


5. Выполните задание 4 с помощью *.value_counts()*

In [ ]:
# Ваш код здесь


6. Найдите среднее и медианное значения числа спален и коек в разрезе по типу собственности в Noord-West.

In [25]:
# Ваш код здесь


7. С помощью сводной таблицы распечатайте среднюю цену размещения каждого типа комнаты а каждом районе.

In [26]:
# Ваш код здесь
